In [39]:
import numpy as np 
import pandas as pd 

In [40]:
df = pd.read_csv("golf-dataset.csv")
X,y  = pre_processing(df)

In [41]:
# no of samples of the data are 
X_size = X.shape[0]

# no of features of the data are 
X_features = X.shape[1]

In [42]:
def pre_processing(df):

 
    X = df.drop([df.columns[-1]], axis = 1)
    y = df[df.columns[-1]]

    return X, y

In [43]:
class_probs = {}
likelihoods = {}
pred_prob = {}    
y_train = y

In [44]:
def calc_class_probs(y,X_size):

    for outcome in np.unique(y):
        outcome_count = sum(y == outcome)
        class_probs[outcome] = outcome_count / X_size
        print("print class_prob",class_probs)        
        

In [45]:
calc_class_probs(y,X_size)

print class_prob {'No': 0.3076923076923077}
print class_prob {'No': 0.3076923076923077, 'Yes': 0.6923076923076923}


In [46]:
def calc_likelihoods(X,y):


        features = list(X.columns)
        for feature in features:
            for outcome in np.unique(y):
                outcome_count = sum(y == outcome)
                feat_likelihood = X[feature][y[y == outcome].index.values.tolist()].value_counts().to_dict()
                for feat_val, count in feat_likelihood.items():
                    likelihoods[feature][str(feat_val) + '_' + outcome] = count/outcome_count
        print("print likelihoods",likelihoods)
                    

In [47]:
def fit(X, y):
    
        features = list(X.columns)
        X_train = X
        y_train = y
        train_size = X.shape[0]
        num_feats = X.shape[1]

        for feature in features:
            likelihoods[feature] = {}
            pred_prob[feature] = {}

            for feature_val in np.unique(X_train[feature]):
                pred_prob[feature].update({str (feature_val): 0})

                for outcome in np.unique(y_train):
                    likelihoods[feature].update({str (feature_val)+'_'+outcome:0})
                    class_probs.update({outcome: 0})

        calc_class_probs(y_train,train_size)
        calc_likelihoods(X_train,y_train)

In [48]:
 # first five rows of dataframe
X2 = X.iloc[0:10]
y2 = y.iloc[0:10]
fit(X2,y2)

print class_prob {'No': 0.4, 'Yes': 0}
print class_prob {'No': 0.4, 'Yes': 0.6}
print likelihoods {'Outlook': {'Overcast_No': 0, 'Overcast_Yes': 0.3333333333333333, 'Rainy_No': 0.75, 'Rainy_Yes': 0.16666666666666666, 'Sunny_No': 0.25, 'Sunny_Yes': 0.5}, 'Temp': {'Cool_No': 0.25, 'Cool_Yes': 0.5, 'Hot_No': 0.5, 'Hot_Yes': 0, 'Mild_No': 0.25, 'Mild_Yes': 0.5}, 'Humidity': {'High_No': 0.75, 'High_Yes': 0.3333333333333333, 'Normal_No': 0.25, 'Normal_Yes': 0.6666666666666666}, 'Windy': {'False_No': 0.5, 'False_Yes': 0.8333333333333334, 'True_No': 0.5, 'True_Yes': 0.16666666666666666}}


In [49]:
def predict(X_test):

        """ Calculates Posterior probability P(c|x) """
        features = list(X.columns)
        results = []
        X_test = np.array(X_test)
        for i in X_test:
            probs_outcome = {}
            for b in np.unique(y_train):
                probabilty = class_probs[b]
                likelihood = 1
                evidence = 1

                for x, feature_val in zip(features, i):
                    likelihood *= likelihoods[x][str(feature_val) + '_' + b]
                output_class = (likelihood * probabilty)

                probs_outcome[b] = output_class

            result = probs_outcome
            results.append(result)

        return np.array(results)

In [50]:
X_test = np.array([['Sunny','Mild', 'High', 'True']])
probs = predict(X_test)
print (probs)
if (probs[0]["No"] > probs[0]["Yes"]):
    print("No")
elif (probs[0]["Yes"] > probs[0]["No"]):
    print("Yes")


[{'No': 0.009375000000000001, 'Yes': 0.008333333333333333}]
No
